<div style="width:100%; background-color: #000041"><a target="_blank\" href="http://university.yugabyte.com\"><img src="assets/YBU_Logo.webp" /></a></div>

# Lab Requirements and Setup

This lab consists of several Jupyter notebooks. The notebooks run in Gitpod. [Gitpod](https://www.gitpod.io/) facilitates runtime environments where a YugabyteDB database cluster can be deployed. Read the following instructions for requirements and setup of the Gitpod environment.

## About Jupyter notebooks
You will use a Jupyter notebook in this lab to run commands to assign environmental variables and Cassandra Query Language commands for the Yugabyte Cloud Query Language known as YCQL. 

There are two types of cells: markdown and code. This is a markdown cell.

You run a code cell by simply selecting the play icon in the cell's left gutter. For code cells, you can modify the code for execution. Certain labs contain challenges or experiments that require you to do just that - modify a code cell and re-run it!

### Requirements
Here are the requirements for this lab:
- Launch using a gitpod workspace
- Run a three node, YugabyteDB cluster using `yb-ctl`

> Note
>  
> Although a three node cluster is up and running, Gitpod does not support visiting loopback addresses over a web ui, even if exposed on a different port.
> 127.0.0.1 is the only web user interfaces. To see all available ports in Gitpod, in the terminal, run `gp ports list`.

#### Notebook keyboard shortcuts
The Jupyter extension for Gitpod supports the following keyboard shortcuts:
| Keystroke | Description |
|--|--|
| ESC | Change the cell mode |
| A | Add a cell above |
| B | Add a cell below |
| J or down arrow key |  Change a cell to below | 
| K or up arrow key | Change a cell to above | 
| Ctrl+Enter | Run the currently selected cell |
| Shift+Enter | Run the currently selected cell and insert a new cell immediately below (focus moves to new cell) |
| Alt+Enter | Run the currently selected cell and insert a new cell immediately below (focus remains on current cell) |
| dd | Delete a selected cell |
| z | Undo the last change | 
| M | switch the cell type to Markdown | 
| Y | switch the cell type to code |
| L | Enable/Disable line numbers |
```


## Setup steps
Here are the steps to setup this lab:
- Install missing dependencies and restart the notebook
- Create the notebook variables
- Create the `ks_ybu` database

### Install missing dependencies and restart the notebook
Run the following cell to ensure that the notebook dependencies are available to the notebook. 

### Create the notebook variables 

> IMPORTANT!
> 
> Do NOT skip running this cell. 
> 

The following Python cell creates and stores variables that all the notebooks in this lab will use. You can view these variables in the Jupyter tab.

- To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.
- Verify the accuracy of the output values

In [ ]:
# Env variables for Notebook
import os

# read env_vars.env
env_vars = !cat env_vars.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value
 

# Comment out Local 
MY_YB_PATH=os.environ.get('MY_YB_PATH_LOCAL')
MY_GITPOD_WORKSPACE_URL=os.environ.get('MY_GITPOD_WORKSPACE_URL_LOCAL')
MY_SUDO=os.environ.get('MY_SUDO')

# Gitpod specific
#MY_YB_PATH=os.environ.get('MY_YB_PATH')
##MY_GITPOD_WORKSPACE_URL=os.environ.get('GITPOD_WORKSPACE_URL')

# env_vars defines the following
MY_DB_NAME=os.environ.get('MY_DB_NAME')
MY_HOST_IPv4_01=os.environ.get('MY_HOST_IPv4_01')
MY_HOST_IPv4_02=os.environ.get('MY_HOST_IPv4_02')
MY_HOST_IPv4_03=os.environ.get('MY_HOST_IPv4_03')
MY_TSERVER_WEBSERVER_PORT=os.environ.get('MY_TSERVER_WEBSERVER_PORT')
MY_DATA_DDL_FILE=os.environ.get("MY_DATA_DDL_FILE")
MY_DATA_DML_FILE=os.environ.get("MY_DATA_DML_FILE")
MY_UTIL_FUNCTIONS_FILE=os.environ.get("MY_UTIL_FUNCTIONS_FILE")
MY_UTIL_YBTSERVER_METRICS_FILE=os.environ.get("MY_UTIL_YBTSERVER_METRICS_FILE")

# Current directory of project and related child folders
MY_NOTEBOOK_DIR=os.getcwd()
MY_NOTEBOOK_DATA_FOLDER=MY_NOTEBOOK_DIR +'/data'
MY_NOTEBOOK_UTILS_FOLDER=MY_NOTEBOOK_DIR + '/utils'

# Store the note book values for other notebooks to use

%store MY_DB_NAME
%store MY_YB_PATH
%store MY_GITPOD_WORKSPACE_URL
%store MY_HOST_IPv4_01
%store MY_HOST_IPv4_02
%store MY_HOST_IPv4_03
%store MY_NOTEBOOK_DIR
%store MY_TSERVER_WEBSERVER_PORT
%store MY_NOTEBOOK_DATA_FOLDER
%store MY_NOTEBOOK_UTILS_FOLDER
%store MY_DATA_DDL_FILE
%store MY_DATA_DML_FILE
%store MY_UTIL_FUNCTIONS_FILE
%store MY_UTIL_YBTSERVER_METRICS_FILE

# YCQL Shell Commands
The following commands are YCQL shell commands. Using `ucqlsh`, you can execute YCQL statements for zero or more keyspaces. 

By default, YugabyteDB and the YCQL API have a consistency level of QUORUM. 

In [ ]:
%%bash -s "$MY_YB_PATH"   # Shell Commands
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "CONSISTENCY"

YCQL is compatible with CQL 3.4.2.

In [ ]:
%%bash -s "$MY_YB_PATH"   # Shell Commands
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "show version"  

In [ ]:
%%bash -s "$MY_YB_PATH"   # Shell Commands
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "select cql_version from system.local;"  

`show host` details the host connection.

In [ ]:
%%bash -s "$MY_YB_PATH"   # Shell Commands
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "show host"  

`decribe kespaces`  returns the names of all keyspaces.

In [ ]:
%%bash -s "$MY_YB_PATH"   # Shell Commands
YB_PATH=${1}
cd $YB_PATH

#./bin/ycqlsh --execute "describe keyspaces"  
./bin/ycqlsh --execute "help describe"  


`describe tables` returns the names of all tables in the current keyspace, or in all keyspacces.

In [ ]:
%%bash -s "$MY_YB_PATH"   # Shell Commands
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "desc tables"

The following queries the roles table. `cassandra` is a default role. 

In [ ]:
%%bash -s "$MY_YB_PATH"   # Shell Commands
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "select * from system_auth.roles;"  

## Create the `ks_ybu` keyspace
Run the following cells to connect to the YugabyteDB cluster using `ycqlsh`. Then, complete the following tasks:
- Create the `ks_ybu` keyspace 
- Create the `tbl_employees` table
- Describe the  `tbl_employees` table

In [ ]:
%%bash -s "$MY_YB_PATH"   # Create the keyspace, ks_ybu, and table, employees
YB_PATH=${1}
cd $YB_PATH


./bin/ycqlsh --execute \"
  create keyspace if not exists ks_ybu;
  "

Confirm the keysapce creation.

In [ ]:
%%bash -s "$MY_YB_PATH"   # Create the keyspace, ks_ybu, and table, employees
YB_PATH=${1}
cd $YB_PATH


./bin/ycqlsh --execute "describe keyspace ks_ybu;
  "

### Create the `tbl_employees` table

If the table already exists, the command will drop the table.

In [ ]:
%%bash -s "$MY_YB_PATH"   # Create the keyspace, ks_ybu, and table, employees
YB_PATH=${1}
cd $YB_PATH


./bin/ycqlsh --execute "
  drop table if exists ks_ybu.tbl_employees;
  "

Create the table, `tbl_employees`. 

> Note: A primary key is required for all user tables in a keysapce.

In [ ]:
%%bash -s "$MY_YB_PATH"   # Create the keyspace, ks_ybu, and table, employees
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
  create table ks_ybu.tbl_employees ( 
    id int, 
    full_name text, 
    email text,
    year int static,
    primary key (id, full_name) 
    );
  "

Describe `tbl_employees`. 

> Question:
>   
> What is different about the table description and the create table statement?

In [ ]:
%%bash -s "$MY_YB_PATH"   # Create the keyspace, ks_ybu, and table, employees
YB_PATH=${1}
cd $YB_PATH


./bin/ycqlsh --execute "desc ks_ybu.tbl_employees "


> Answer:
>   
> The description of the table reveals that the primary key consists of two key columns. The first key column is `id`. The second key column is `full_name`. `id` is the partition key for the table. The clustering key column is `full_name`. Ascending is the order of the clustering key column.

#### Write to the table employees with `insert`

Insert rows into the `tbl_employees` table.

In [ ]:
%%bash -s "$MY_YB_PATH"   # Populate the employees table
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
  insert into ks_ybu.tbl_employees 
    (id, full_name, email, year) 
    values 
    (1, 'Bruce Wayne', 'batman@yb.com', 2020);
 "

 ./bin/ycqlsh --execute "
  insert into ks_ybu.tbl_employees 
    (id, full_name, email, year) 
    values 
    (1, 'Dick Grayson', 'robin@yb.com', 2020);
 "

./bin/ycqlsh --execute "
  insert into ks_ybu.tbl_employees 
     (id, full_name, email, year) 
    values 
    (2, 'Clark Kent', 'superman@yb.com', 2021);
"

./bin/ycqlsh --execute "
  insert into ks_ybu.tbl_employees 
     (id, full_name, email, year) 
    values 
    (3, 'Peter Parker', 'spiderman@yb.com', 2022);
"

### Query the table using `select`
Verify the row insertion. Run the following cells to observe the differences between:
- query all rows of the table 
- query the table with a where-expression predicate that contains the primary key and a equality operator
- query the table with a where-expression predicate that for a range for a regular column

Query all the rows from `tbl_employees` using a`select statement` and a wildcard.

In [ ]:
%%bash -s "$MY_YB_PATH"   # Data search
YB_PATH=${1}
cd $YB_PATH

# Read all columns from the employees table
./bin/ycqlsh --execute "
    select * from ks_ybu.tbl_employees;
    "

Query the table using a where-expression predicate that contains the primary key and a equality operators.

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Find an employee id = 3
./bin/ycqlsh --execute "
  select *
  from ks_ybu.tbl_employees
  where id = 1 and full_name='Dick Grayson';
"

Query the table using where-expression predicate that specifies a range for a regular column.

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
  select * from ks_ybu.tbl_employees 
  where  year > 2020 and year < 2022;
"

### Observe the upsert behavior
The following update statement exhibits upsert behavior when there is no row to update based on the where clause predicate.

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
 update ks_ybu.tbl_employees
 set email='captainamerica@yb.com'
   , year = 2022
 where id = 4 
   and full_name = 'Steven Rogers'
  "

Confirm the new row.

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
 select * from ks_ybu.tbl_employees where id = 4;
  "

## todo DELTE Behavior


single-row deleteion

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  delete from ks_ybu.tbl_employees where id = 6;
"

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
 select * from ks_ybu.tbl_employees;
  "

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  create table ks_ybu.tbl1 (col_pk int, col_ck1 text, col_reg int, primary key (col_pk, col_ck));
  "

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  insert into ks_ybu.tbl1 (col_pk , col_ck , col_reg ) values ( 1, 'green', 2022);
  insert into ks_ybu.tbl1 (col_pk , col_ck , col_reg ) values ( 1, 'red', 2022);
  insert into ks_ybu.tbl1 (col_pk , col_ck , col_reg ) values ( 2, 'red', 2022);
  insert into ks_ybu.tbl1 (col_pk , col_ck , col_reg ) values ( 2, 'green', 2022);
  insert into ks_ybu.tbl1 (col_pk , col_ck , col_reg ) values ( 3, 'blue', 2022);
  insert into ks_ybu.tbl1 (col_pk , col_ck , col_reg ) values ( 3, 'red', 2022);
  "

### single row, PK

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  delete from ks_ybu.tbl1 where col_pk = 1 and col_ck = 'green';
  "

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  select * from ks_ybu.tbl1;
  "

### multi-row

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  delete from ks_ybu.tbl1 where col_pk=1 and col_ck > 'green'
  "

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  select * from ks_ybu.tbl1;
  "

### delete a column value?


In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  delete col_reg from ks_ybu.tbl1 where col_pk = 3 and col_ck = 'blue';
  "

#### delete if

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  delete from ks_ybu.tbl1 where col_pk = 2 and col_ck = 'green' if col_ck > 2021;
  "

## Import data from SQL scripts
In this section, you will create a new keyspace, `ks_ybu`, where you will import sql scripts. Run the next cell, to create the keyspace, `ks_ybu`.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" 
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

# Drop table if exisits so that namespace can be created
./bin/ycqlsh --execute "drop table if exists ks_ybu.employees;"


# Drop keyspace if exists
./bin/ycqlsh --execute "drop keyspace if exists "${DB_NAME}";"


 
# Create keyspace, ks_ybu
./bin/ycqlsh --execute "create keyspace "${DB_NAME}";" 



# List keyspaces, validate keyspace creation
./bin/ycqlsh --execute "describe "${DB_NAME}";"

To verify that the keyspace, ks_ybu, was created successfully, you can use the DESCRIBE shell command, also abbreviated as DESC.

### Load DDL and Data from File
You will run the following cell in order to assign the file paths of the DML and DDL sql scripts. Once the DDL file creates the tables, the DML file will populate the tables with data.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_DATA_FOLDER" "$MY_DATA_DDL_FILE" "$MY_DATA_DML_FILE"   
# Wishlist
YB_PATH=${1}
DB_NAME=${2}
DATA_FOLDER=${3}
DATA_DDL_FILE=${4}
DATA_DML_FILE=${5}

# ls $DATA_FOLDER

WISHLIST_DDL_PATH=${DATA_FOLDER}/${DATA_DDL_FILE}
WISHLIST_DML_PATH=${DATA_FOLDER}/${DATA_DML_FILE}

cd $YB_PATH

# DDL file
./bin/ycqlsh -k ${DB_NAME} -f ${WISHLIST_DDL_PATH} 
sleep 1;

# # DML file
./bin/ycqlsh -k ${DB_NAME} -f ${WISHLIST_DML_PATH} 
sleep 1;

# # Describe relations
./bin/ycqlsh --execute "describe tables"

In the preceeding import commands, you ran the YCQL commands in the YCQL shell to connect to the ks_ybu keyspace and execute the respective sql scripts. Note that the DESCRIBE TABLES shell command displays the schemas of each table in the keyspace, `ks_ybu`. Note that the DESCRIBE shell command can be used to describe keyspaces, tables, indexes, and sequences as well. For more information about the use of DESCRIBE, use the HELP DESCRIBE command.

### Verify the tables were seeded
Run the following cell to verfy each of the three tables 

In [ ]:
%%bash -s "$MY_YB_PATH"   # Verify data population
YB_PATH=${1}
cd $YB_PATH

# Read data in the tbl_wishlists_by_category
./bin/ycqlsh --execute "
  select * from ks_ybu.tbl_products_by_category;
"  

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Read data in the tbl_wishlists_by_user 
./bin/ycqlsh --execute "
  select * from ks_ybu.tbl_wishlists_by_user;
" 

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Read data in the tbl_products_by_wishlist
./bin/ycqlsh --execute "
  select * from ks_ybu.tbl_products_by_wishlist;
"  

In [ ]:
%%bash -s "$MY_YB_PATH"   # Drop tables
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
  drop table if exists ks_ybu.tbl_products_by_wishlist;
" 

./bin/ycqlsh --execute "
  drop table if exists ks_ybu.tbl_products_by_category;
" 

./bin/ycqlsh --execute "
  drop table if exists ks_ybu.tbl_wishlists_by_user;
" 

./bin/ycqlsh --execute "
  drop keyspace if exists ks_ybu;
" 

---
# All done!
In this lab, you completed the following:

- Setup
  - Created the `ks_ybu` keyspace with `ycqlsh`
  - Created tables and loaded data using DDL and DML scripts

In [ ]:
%%bash
gp open 02_YCQL_Intermediate.ipynb

### Slides:
Create the keyspace.

In [ ]:
%%bash -s "$MY_YB_PATH"   # Drop tables
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "create keyspace ks_catalog;" 


In [ ]:
%%bash -s "$MY_YB_PATH"   # Drop tables
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "drop table if exists ks_ybu.tbl_music;" 


In [ ]:
%%bash -s "$MY_YB_PATH"   # Drop tables
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "create table if not exists ks_ybu.tbl_music (artist text primary key, song text, year int);" 


In [ ]:
%%bash -s "$MY_YB_PATH"   # Drop tables
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "create table if not exists ks_ybu.tbl_music (artist text, song text, year int, primary key ((artist, song)));" 


In [ ]:
%%bash -s "$MY_YB_PATH"   # Drop tables
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "drop table if exists ks_ybu.tbl_music;"

./bin/ycqlsh --execute "
    create table if not exists ks_ybu.tbl_music (
        artist text, 
        song text, 
        year int, 
        primary key ((artist), song, year ) 
        )
    with transactions = {'enabled': 'true'}
    ;
"

In [ ]:
%%bash -s "$MY_YB_PATH"   # Drop tables
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
 describe ks_ybu.tbl_music;
 "

In [ ]:
%%bash -s "$MY_YB_PATH"   # Drop tables
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
insert into ks_ybu.tbl_music ( artist, song, year) values ('Beatles','A',1966);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Kate Bush','A',1988);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Radiohead','A',1997);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Harry Styles','A',2022);
"

In [ ]:
%%bash -s "$MY_YB_PATH"   # Drop tables
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "drop table if exists ks_ybu.tbl_music;"

./bin/ycqlsh --execute "create table if not exists ks_ybu.tbl_music (artist text, song text, year int, primary key ((artist) , song , year) );" 


In [ ]:
%%bash -s "$MY_YB_PATH"   # Drop tables
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
insert into ks_ybu.tbl_music ( artist, song, year) values ('Beatles','A',1966);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Beatles','B',1968);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Beatles','C',1968);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Beatles','D',1966);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Beatles','E',1966);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Harry Styles','E',2021);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Harry Styles','C',2021);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Harry Styles','D',2020);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Harry Styles','B',2022);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Harry Styles','A',2022);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Kate Bush','E',1985);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Kate Bush','A',1988);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Kate Bush','D',1985);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Kate Bush','B',1988);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Kate Bush','C',1985);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Radiohead','B',1998);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Radiohead','A',1997);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Radiohead','C',2020);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Radiohead','D',1998);
insert into ks_ybu.tbl_music ( artist, song, year) values ('Radiohead','E',1997);
"

In [ ]:
%%bash -s "$MY_YB_PATH"   # Drop tables
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "select * from ks_ybu.tbl_music;"

In [ ]:
MY_OBJECT_NAME="idx_music_jsonb"
%store MY_OBJECT_NAME
print(MY_OBJECT_NAME)

In [ ]:
%%bash -s "$MY_HOST_IPv4_01" "$MY_HOST_IPv4_02" "$MY_HOST_IPv4_03" --out MY_HOST_IPv4

HOST_IPv4_01=${1}
HOST_IPv4_02=${2}
HOST_IPv4_03=${3}

# change the hosts for different tablet leaders
# MY_HOST_IPv4=$HOST_IPv4_01
MY_HOST_IPv4=$HOST_IPv4_02
#MY_HOST_IPv4=$HOST_IPv4_03

echo ${MY_HOST_IPv4}

In [ ]:
%store MY_HOST_IPv4
print(MY_HOST_IPv4)


In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLE_ID

OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")

MY_URL="http://${HOST_IPv4}:8200/metrics"

MY_INDEX_TABLE_ID=`curl -s ${MY_URL} | jq -r 'limit(1;  .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="'$OBJECT_NAME'") |  .attributes.table_id) '`

echo ${MY_INDEX_TABLE_ID} 


In [ ]:
%store MY_INDEX_TABLE_ID
print(MY_INDEX_TABLE_ID)

In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLET_ID
OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")

MY_URL="http://${HOST_IPv4}:8200/metrics"


INDEX_TABLET_ID=`curl -s ${MY_URL} | jq --raw-output ' .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="'$OBJECT_NAME'") | {tablet_id: .id, metrics: .metrics[] | select(.name == ("is_raft_leader") ) | select(.value == 1) } | select(.tablet_id) | {tablet_id} | .tablet_id '`

echo ${INDEX_TABLET_ID}

In [ ]:
%store MY_INDEX_TABLET_ID
print(MY_INDEX_TABLET_ID)

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_HOST_IPv4_01" "$MY_INDEX_TABLE_ID"  # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}

INDEX_TABLE_ID=$( echo "${4}" | tr -d " ")

cd $YB_PATH

./bin/yb-admin -init_master_addrs ${HOST_IPv4}:7100 flush_table_by_id ${INDEX_TABLE_ID} 600


In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_HOST_IPv4_01" "$MY_INDEX_TABLE_ID" "$MY_INDEX_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}
INDEX_TABLE_ID=$( echo "${4}" | tr -d " ")
INDEX_TABLET_ID=$( echo "${5}" | tr -d " ")

cd $YB_PATH
cd bin/

TABLE_ID_PATH=/Users/seth/yugabyte-data/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${INDEX_TABLE_ID}/tablet-${INDEX_TABLET_ID}
ls -l  ${TABLE_ID_PATH}

./sst_dump --command=scan --file=${TABLE_ID_PATH} --output_format=decoded_regulardb

# ./sst_dump --command=scan --file=${TABLE_ID_PATH} --show_properties
# -command=scan --read_num=5

## Part 5 | Built-in functions

date-time

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_HOST_IPv4_01" "$MY_INDEX_TABLE_ID" "$MY_INDEX_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}
INDEX_TABLE_ID=$( echo "${4}" | tr -d " ")
INDEX_TABLET_ID=$( echo "${5}" | tr -d " ")

cd $YB_PATH
cd bin/

TABLE_ID_PATH=/Users/seth/yugabyte-data/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${INDEX_TABLE_ID}/tablet-${INDEX_TABLET_ID}
ls -l  ${TABLE_ID_PATH}

./sst_dump --command=scan --file=${TABLE_ID_PATH} --output_format=decoded_regulardb

# ./sst_dump --command=scan --file=${TABLE_ID_PATH} --show_properties
# -command=scan --read_num=5

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  select col_pk, totimestamp(now()),todate(now()), dateof(now()) from ks_ybu.tbl1 


"

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  select col_pk, col_ck, col_reg, currentdate(), currenttime(), currenttimestamp(), todate(now()) from ks_ybu.tbl1 


"

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  select partition_hash(artist,song) from ks_ybu.tbl_music;


"

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Search a range of values
./bin/ycqlsh --execute "
  select partition_hash(col_pk),col_pk, partition_hash('AAAA'),col_ck, col_reg, currentdate(), currenttime(), currenttimestamp(), todate(now()) from ks_ybu.tbl1 


"

---
JSON with tbl_music


In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "alter table tbl_music add tours jsonb;"

In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "select * from ks_ybu.tbl_music;"

insert into tbl_music ( artist, song, year, tours) 
 values (
    'Harry Styles',
    'F',
    2023,
    '{ \"Events\": [{ \"Date\": \"2023-01-29\", \"Venue\": \"Kia Forum\",\"Capacity\": 50000,\"Members\": {\"Piano\": \"Charlotte Clark",  \"Vocalist\": \"Ny Oh\"} }, { \"Date\": \"2023-06-17\", \"Venue\": \"Wembley Stadium\", \"Capacity\": 75000, \"Members\": { \"Vocalist\": \"Ny Oh\", \"Bass Guitar\": \"Pauli Lovejoy\" }}] }'

Insert

In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
 insert into tbl_music ( artist, song, year, tours) 
 values (
    'Harry Styles',
    'F',
    2023,
    '{ \"Events\": [  { \"Date\": \"2023-01-29\",
          \"Venue\": \"Kia Forum\",
          \"Capacity\": 50000,
          \"Members\": {\"Piano\": \"Charlotte Clark\",
              \"Vocalist\": \"Ny Oh\"} 
              },
        { \"Date\": \"2023-06-17\", \"Venue\": \"Wembley Stadium\", \"Capacity\": 75000, \"Members\": { \"Vocalist\": \"Ny Oh\", \"Bass Guitar\": \"Pauli Lovejoy\" }}] }' 
    );
"

Update | add- key-value pair
` set sku_details->'rating' = '3'`

In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  update tbl_music
    set tours->'Events'->0->>'City' = '\"Inglewood\"'
    where  artist = 'Harry Styles'
    and song = 'F'
    and year = 2023;
    "


los angelese

In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  update tbl_music
    set tours->'Events'->0->>'City' = '\"Los Angeles\"'
    where  artist = 'Harry Styles'
    and song = 'F'
    and year = 2023;
    "


In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
    select * from tbl_music
    where artist = 'Harry Styles'
    and year = 2023;"

In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  update tbl_music
    set tours = '{ \"Events\": [{ \"Date\": \"2023-01-29\", \"Venue\": \"Kia Forum\",\"Capacity\": 50000,\"Members\": {\"Piano\": \"Charlotte Clark\",  \"Vocalist\": \"Ny Oh\"} }, { \"Date\": \"2023-06-17\", \"Venue\": \"Wembley Stadium\", \"Capacity\": 75000, \"Members\": { \"Vocalist\": \"Ny Oh\", \"Bass Guitar\": \"Pauli Lovejoy\" }}] }'
    where  artist = 'Harry Styles'
    and song = 'F'
    and year = 2023;
    "


In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
    select * from tbl_music
    where artist = 'Harry Styles'
    and year = 2023;"

In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
select artist,
  tours->'Events'->0->>'Date' as date_1,
  tours->'Events'->0->>'Venue' as venue_1,
  tours->'Events'->0->>'Capacity' as capacity_1,
  tours->'Events'->0->'Members' as members_1
from tbl_music  
where artist = 'Harry Styles'
    and year = 2023
    and song = 'F'  ;
    "

In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
select artist,
  tours->'Events'->0->>'Date' as date_1,
  tours->'Events'->0->>'Venue' as venue_1,
  tours->'Events'->0->>'Capacity' as capacity_1,
  tours->'Events'->0->'Members'->>'Piano' as Piano_1,
  tours->'Events'->0->'Members'->>'Vocalist' as Vocalist_1
from tbl_music
where tours->'Events'->0->>'Date' = '2023-01-29';
    "

In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
explain select artist,
  tours->'Events'->0->>'Date' as date_1,
  tours->'Events'->0->>'Venue' as venue_1,
  tours->'Events'->0->>'Capacity' as capacity_1,
  tours->'Events'->0->'Members'->>'Piano' as Piano_1,
  tours->'Events'->0->'Members'->>'Vocalist' as Vocalist_1
from tbl_music
where tours->'Events'->0->>'Date' = '2023-01-29';
    "

In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
    drop index if exists idx_music_jsonb
    ;
    "

In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
    create index if not exists idx_music_jsonb
     on tbl_music (
        tours->'Events'->0->>'Date',
        tours->'Events'->0->>'Venue',
        tours->'Events'->0->>'Capacity', 
        tours->'Events'->0->'Members'->>'Piano', 
        tours->'Events'->0->'Members'->>'Vocalist'
        )
    ;
    "

In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_DB_NAME"  # Query for id
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
explain select artist, song, year,
  tours->'Events'->0->>'Date' as date_1,
  tours->'Events'->0->>'Venue' as venue_1,
  tours->'Events'->0->>'Capacity' as capacity_1,
  tours->'Events'->0->'Members'->>'Piano' as Piano_1,
  tours->'Events'->0->'Members'->>'Vocalist' as Vocalist_1
from tbl_music
where tours->'Events'->0->>'Venue' = '"?"';
    "

grep IMPORTANT data.csv | cassandra-loader -f stdin -h 1.2.3.4 -schema "test.itest(a, b)"

mysql -h 192.168.128.1 -e 'select * from db1.vw_music_json' | cassandra-loader -f stdin -h 127.0.0.1 -schema "ks_ybu.tbl_music (artist, song, year, tours)"

In [ ]:
%%bash
HOST_MYSQL=192.168.128.1
HOST_YCQL=127.0.0.1
VW_MYSQL="select artist, song, year, tours from db1.vw_music_json;"
SCHEMA_YCQL="ks_ybu.tbl_music (artist, song, year, tours)"

echo ${VW_MYSQL} \ 
  $SCHEMA_YCQL
